In [1]:
# Documentation link: https://developers.arcgis.com/python/guide/cloning-content/

# Connect to GIS: 
import arcgis
from arcgis.gis import GIS

survey_form_item_id = '<Item ID of the Form Item>'

source_Username = '<Origin Portal Username>'
source_Password = '<Origin Portal Password>'

target_Username = '<Destination Portal Username>'
target_Password = '<Destination Portal Password>'

# Existing survey environment (ArcGIS Online or ArcGIS Enterprise Portal)
source = GIS(username=source_Username, password=source_Password)

# Destination survey environment (ArcGIS Online or ArcGIS Enterprise Portal)
target = GIS(username=target_Username, password=target_Password)

**Note** if you are using ArcGIS Enterprise with a domain certificate use `verify_cert=False` when making your GIS connection. For more information on additional parameters when making a GIS connection see <a href="https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html#gis" target="_blank">this documentation</a>, as well as the <a href="https://developers.arcgis.com/python/guide/working-with-different-authentication-schemes/" target="_blank">working with different authentication schemes documentation</a>.

In [2]:
# Identify the Form Item
form_item = arcgis.gis.Item(source,survey_form_item_id)
survey_form = [form_item]

# For more information on the related_items function visit: https://developers.arcgis.com/python/api-reference/arcgis.gis.toc.html?#arcgis.gis.Item.related_items
# Grab the Feature Service associated with the Survey
related_items = form_item.related_items('Survey2Service','forward')

# Grab the additional items that have a relationship with the Survey
additional_items = form_item.related_items('Survey2Data','forward')

# Merge the all related content into the same list
all_items = related_items + additional_items + survey_form
all_items

[<Item title:"DogsConnect" type:Feature Layer Collection owner:ZSutherby_Survey123>,
 <Item title:"DogsConnect" type:Web Map owner:ZSutherby_Survey123>,
 <Item title:"DogsConnect_sampleTemplateIndividual" type:Microsoft Word owner:ZSutherby_Survey123>,
 <Item title:"DogsConnect" type:Form owner:ZSutherby_Survey123>]

In [3]:
# Define Survey Manager: 
survey_manager = arcgis.apps.survey123.SurveyManager(source)

# Grab a survey using the Item ID:
survey_by_id = survey_manager.get(survey_form_item_id)

# Grab the survey properties (for the folder ID): 
survey_folder = survey_by_id.properties['ownerFolder']
print (survey_folder)

9a386df99f684c9a8a9efa7211e01352


In [4]:
# Identify User Acccount
usr = arcgis.gis.User(source,source_Username)

# List all folders owned by the user
full_folder = usr.folders

# Identify the folder associated with the survey
#print([f for f in full_folder if f['id'] == survey_folder][0])
fldr = next((f for f in full_folder if f['id'] == survey_folder), 0)

In [5]:
# Create a new folder with the same name as the source folder to clone the contents to
target.content.create_folder(folder=fldr['title'])

# Clone contents to the new GIS
cloned_items = target.content.clone_items(items=all_items, folder=fldr['title'])
cloned_items

[<Item title:"DogsConnect" type:Feature Layer Collection owner:portaladmin>,
 <Item title:"DogsConnect_sampleTemplateIndividual" type:Microsoft Word owner:portaladmin>,
 <Item title:"DogsConnect" type:Web Map owner:portaladmin>,
 <Item title:"DogsConnect" type:Form owner:portaladmin>]